# <b> Project 1

In this project, there will be three parts to analyze Dataframe, a sub-library of pandas. 

In [67]:
import pandas as pd
arr = [{"1","2"}]
arr1 = 1
a = pd.DataFrame(data = arr)
b = pd.Series(data = arr1)


In [68]:
type(a.iloc[0][1])

str

The following BNF grammar definition of Pandas:


\begin{eqnarray*}
\textit{stmt}& ::= &\textbf{Assign(expr* targets, stmt value))} \\
\\
\\
\textit{expr}& ::= &\textbf{DataFrame(expr)} \\
                 &  |  &\textbf{Series(expr* value)} \\
                 &  |  &\textbf{ Num(object n)} \\
                 &  |  &\textbf{ Numpy(expr* elts)} \\
                 &  |  &\textbf{ Str(string s)} \\
                 &  |  &\textbf{ Name(identifier id, expr_context ctx)} \\
                 &  |  &\textbf{ Dict(expr* keys,  expr* values)} \\
                 &  |  &\textbf{List(expr* elts, expr_context ctx)} \\
                 &  |  &\textbf{Tuple(expr* elts, expr_context ctx)} \\
                 &  |  &\textbf{Slice(expr* value)} \\
                 &  |  &\textbf{Call(expr func, expr* args, keyword* keywords)} \\
                 &  |  &\textbf{Compare(expr left, cmpop* ops, expr* comparators)} \\
                 \\
                 \\
\textit{cmpop}& ::= &\textbf{Eq | NotEq | Lt | LtE | Gt | GtE } \\
\\
\textit{keyword arguments supplied to call:}\\
\textit{keyword}& ::= &\textbf{(identifier? arg, expr value) } \\
\end{eqnarray*}


In [ ]:
#
import ast 
import pandas as pd
"""class is_simple(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        return all(results)

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_BinOp(self, node):
        return self.visit(node.left) and self.visit(node.right)
    
    def visit_Num(self, node):
        return True
    
a = ast.parse('5 + 5 - 7 // 4 * 4')
print(is_simple().visit(a))

a = ast.parse('x + 5')
print(is_simple().visit(a))"""

'''Below is an implementation of an algorithm that identifies input statement is 
using panads DataFrame data structrue only'''
class is_DataFrame(ast.NodeVisitor):
    